In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import pretty_midi
import librosa
import librosa.display
import gc
from sklearn.preprocessing import StandardScaler
import warnings
from collections import Counter
from torch.utils.data import Dataset
import torch
from pympler import asizeof



import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from sklearn.preprocessing import StandardScaler

from Preprocessing import *
#from ExtractGenre import *
from CNN_ExtractGenre import *
from PolyphonicPreprocessing import *
import Util as Util

import DatasetLoader as DL



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
GenreMapping = {'metal': 0, 'disco': 1, 'classical': 2, 'hiphop': 3, 'jazz': 4,
          'country': 5, 'pop': 6, 'blues': 7, 'reggae': 8, 'rock': 9}

In [17]:
Path = os.path.realpath('clean_midi/Eminem/Bad Meets Evil.mid')
mid = mido.MidiFile(Path)
#print(mid)

In [32]:
test = torch.load('Dataset_Percussion.pt')

In [40]:
torch.sum(test['Percussion'][11]['Bars'][1])

tensor(2)

In [4]:
asizeof.asizeof(test)

NameError: name 'test' is not defined

In [41]:
test['Percussion'][11]

{'SongName': ("I'm So in Love With You", "I'm So in Love With You"),
 'Bars': (tensor(indices=tensor([[64, 64, 64],
                         [12, 14, 15]]),
         values=tensor([1, 1, 1]),
         size=(128, 16), nnz=3, dtype=torch.int32, layout=torch.sparse_coo),
  tensor(indices=tensor([[64, 64],
                         [12, 14]]),
         values=tensor([1, 1]),
         size=(128, 16), nnz=2, dtype=torch.int32, layout=torch.sparse_coo)),
 'Program': 176,
 'Channel': (10, 10),
 'numBar': (22, 23),
 'Tempo': (128, 128)}

In [44]:
for i in range(128):
   print(torch.sum(test['Bass'][i]['Bars'][0]))

tensor(17)
tensor(11)
tensor(16)
tensor(27)
tensor(16)
tensor(17)
tensor(16)
tensor(18)
tensor(18)
tensor(18)
tensor(17)
tensor(16)
tensor(12)
tensor(17)
tensor(18)
tensor(16)
tensor(17)
tensor(17)
tensor(17)
tensor(11)
tensor(16)
tensor(27)
tensor(16)
tensor(17)
tensor(16)
tensor(18)
tensor(18)
tensor(18)
tensor(17)
tensor(16)
tensor(12)
tensor(17)
tensor(18)
tensor(16)
tensor(17)
tensor(17)
tensor(1)
tensor(19)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(20)
tensor(21)
tensor(22)
tensor(20)
tensor(20)
tensor(21)
tensor(22)
tensor(20)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(1)
tensor(19)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(20)
tensor(21)
tensor(22)
tensor(20)
tensor(20)
tensor(21)
tensor(22)
tensor(20)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(18)
tensor(1)
tensor(19)
tensor(18)
te

In [7]:
polconc = []
for i in range(30, 50):
   for j in range(2):
      polconc.append(test['Piano'][i]['Bars'][j].to_dense())

In [8]:
conc = np.concatenate(polconc, axis = 1)
MonoBarsToMIDI(conc, title = 'test', Instrument = 4)

In [9]:
poltest = PolyphonicPreProcessing(nDir = 30)

100%|██████████| 7/7 [00:00<00:00, 582.02it/s]


In [10]:
polconc = []
for i in range(30, 50):
   for j in range(2):
      polconc.append(poltest['rock'][i]['Bars'][j].to_dense())

In [11]:
conc = np.concatenate(polconc, axis = 2)
poltest['rock'][30]['Program']
print(conc.shape)

(4, 128, 640)


In [12]:
PolyBarsToMIDI(conc, title = 'poltest', Instrument = [113, 25, 1, 41])